In [ ]:
!pip3 install transformers==4.49.0 torch pandas tqdm

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]


## Download all BMW court cases from courtlistener.com

In [2]:
import requests
import json
import time

In [8]:
# Base API endpoint
base_url = "https://www.courtlistener.com/api/rest/v4/search/?q=bmw"

# Headers to make it look like a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Initialize list to store all cases
all_cases = []

# Counter for number of cases collected
total_cases_collected = 0
max_cases = 10000

# Initialize next_url to the base URL
next_url = base_url

# Loop until we've collected enough cases or there are no more pages
while next_url and total_cases_collected < max_cases:
    print(f"Fetching cases from: {next_url}")
    
    # Make the request with a small delay to avoid rate limiting
    time.sleep(0.2)
    response = requests.get(next_url, headers=headers)
    
    # Check if request was successful
    if response.status_code == 200:
        # Parse JSON response
        data = response.json()
        
        # Extract results
        results = data.get('results', [])
        
        # Add these results to our collection
        all_cases.extend(results)
        total_cases_collected = len(all_cases)
        
        print(f"Collected {len(results)} cases, total: {total_cases_collected}")
        
        # Get URL for the next page
        next_url = data.get('next')
        
        # If we've reached the limit, break the loop
        if total_cases_collected >= max_cases:
            break
    else:
        print(f"Failed to retrieve data: Status code {response.status_code}")
        print(response.text)
        break

# Trim to exactly max_cases if we have more
if len(all_cases) > max_cases:
    all_cases = all_cases[:max_cases]

# Save to JSON file
output_file = "bmw_court_cases.json"
with open(output_file, 'w') as f:
    json.dump(all_cases, f)

print(f"Successfully collected {len(all_cases)} BMW court cases and saved to {output_file}")

Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?q=bmw
Collected 20 cases, total: 20
Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?cursor=cz0xMzguMjMwNDcmcz00OTAzNjY1JnQ9byZkPTIwMjUtMDQtMjY%3D&q=bmw
Collected 20 cases, total: 40
Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?cursor=cz0xMjkuMDU0OTImcz0yNzc1ODE1JnQ9byZkPTIwMjUtMDQtMjY%3D&q=bmw
Collected 20 cases, total: 60
Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?cursor=cz0xMjEuODM5NDEmcz03MzE4ODMwJnQ9byZkPTIwMjUtMDQtMjY%3D&q=bmw
Collected 20 cases, total: 80
Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?cursor=cz0xMTQuMjQ1OSZzPTY4OTc2MDAmdD1vJmQ9MjAyNS0wNC0yNg%3D%3D&q=bmw
Collected 20 cases, total: 100
Fetching cases from: https://www.courtlistener.com/api/rest/v4/search/?cursor=cz0xMDYuOTczNTUmcz0yMDU3MjAyJnQ9byZkPTIwMjUtMDQtMjY%3D&q=bmw


KeyboardInterrupt: 

In [3]:
import json
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time

# Load the JSON file
with open("bmw_court_cases.json", "r") as f:
    bmw_cases = json.load(f)

In [15]:
bmw_cases = bmw_cases[:100]

In [19]:
# Define function to fetch case text for a single case
def fetch_case_text(case):
    absolute_url = case["absolute_url"]
    id = absolute_url.replace("/opinion/", "").split("/")[0]
    
    # Headers with user agent to make it look like a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Add delay to avoid rate limiting
        time.sleep(1.0)  # Increased delay to 1 second
        
        response = requests.get("https://www.courtlistener.com/api/rest/v4/opinions/" + id,
                               headers=headers)
        if response.status_code == 200:
            case_data = response.json()
            return id, case_data.get("plain_text", "")
        elif response.status_code == 429:  # Too Many Requests
            # Wait longer and retry once
            print(f"Rate limited for case ID: {id}, waiting 10 seconds before retry")
            time.sleep(10)
            response = requests.get("https://www.courtlistener.com/api/rest/v4/opinions/" + id,
                                   headers=headers)
            if response.status_code == 200:
                case_data = response.json()
                return id, case_data.get("plain_text", "")
            else:
                print(f"Retry failed for case ID: {id}, status code: {response.status_code}")
                return id, ""
        else:
            print(f"Failed to fetch data for case ID: {id}, status code: {response.status_code}")
            return id, ""
    except Exception as e:
        print(f"Error fetching case ID: {id}, error: {str(e)}")
        return id, ""

# Sequential downloads instead of parallel
print(f"Downloading text for {len(bmw_cases)} cases...")
case_id_to_text = {}

# Process each case sequentially with a progress bar
for i, case in tqdm(enumerate(bmw_cases), total=len(bmw_cases), desc="Downloading case texts"):
    case_id, text = fetch_case_text(case)
    if text:
        bmw_cases[i]["text"] = text
        case_id_to_text[case_id] = text

# Save updated cases to JSON file
output_file = "bmw_court_cases_with_text.json"
with open(output_file, 'w') as f:
    json.dump(bmw_cases, f)

print(f"Successfully updated {len(case_id_to_text)} cases with text and saved to {output_file}")


Rate limited for case ID: 7323818, waiting 10 seconds before retry


KeyboardInterrupt: 

In [14]:
bmw_cases[0]["text"]

KeyError: 'text'

In [ ]:
import torch.nn.functional as F
import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Use MPS (Metal Performance Shaders) if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
batch_dict = {k: v.to(device) for k, v in batch_dict.items()}  # Move to MPS
    

# Get the model outputs
with torch.no_grad():  # Disable gradient computation for inference
    outputs = model(**batch_dict)

# Average pool the outputs
embedding = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])

# Normalize the embedding
embedding = F.normalize(embedding, p=2, dim=1)

# normalize embeddings
embeddings = F.normalize(embedding, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())
